# Data Cleaning
In this notebook, we are doing basic data cleaning as well as splitting of the dataset.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import sys
dir_path = "/content/drive/MyDrive/Colab Notebooks/LoanDefaults/"
sys.path.append(dir_path + "utils/")

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, normalize, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression, RFE

from sklearn import set_config
set_config(transform_output="pandas")
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression


from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import joblib
import os

import plot_functions as my
import helper as hl

In [4]:
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 125)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Applications (Main File)

In [5]:
full_loans = pd.read_csv(dir_path + "original_data/application_train.csv")
full_loans.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,351000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.019,-9461,-637,-3648.000,-2120,NaN,1,1,0,1,1,0,Laborers,1.000,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083,0.263,0.139,0.025,0.037,0.972,0.619,0.014,0.000,0.069,0.083,0.125,0.037,0.020,0.019,0.000,0.000,0.025,0.038,0.972,0.634,0.014,0.000,0.069,0.083,0.125,0.038,0.022,0.020,0.000,0.000,0.025,0.037,0.972,0.624,0.014,0.000,0.069,0.083,0.125,0.037,0.021,0.019,0.000,0.000,reg oper account,block of flats,0.015,"Stone, brick",No,2.000,2.000,2.000,2.000,-1134.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,1129500.000,Family,State servant,Higher education,Married,House / apartment,0.004,-16765,-1188,-1186.000,-291,NaN,1,1,0,1,1,0,Core staff,2.000,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311,0.622,NaN,0.096,0.053,0.985,0.796,0.060,0.080,0.035,0.292,0.333,0.013,0.077,0.055,0.004,0.010,0.092,0.054,0.985,0.804,0.050,0.081,0.035,0.292,0.333,0.013,0.079,0.055,0.000,0.000,0.097,0.053,0.985,0.799,0.061,0.080,0.035,0.292,0.333,0.013,0.079,0.056,0.004,0.010,reg oper account,block of flats,0.071,Block,No,1.000,0.000,1.000,0.000,-828.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,135000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010,-19046,-225,-4260.000,-2531,26.000,1,1,1,1,1,0,Laborers,1.000,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.556,0.730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
print("Number of Duplicates:", full_loans.duplicated().sum())

Number of Duplicates: 0


## Changing Data Types

In [7]:
for column in full_loans.columns:
    if not pd.api.types.is_numeric_dtype(full_loans[column]):
        print(f"{column}: dtype: {full_loans[column].dtype}, unique values: {full_loans[column].nunique()}", end="")
        if full_loans[column].nunique() <= 4:
            print(". Categories:", full_loans[column].unique())
        else:
            print(".")

        full_loans[column] = full_loans[column].astype("category")

NAME_CONTRACT_TYPE: dtype: object, unique values: 2. Categories: ['Cash loans' 'Revolving loans']
CODE_GENDER: dtype: object, unique values: 3. Categories: ['M' 'F' 'XNA']
FLAG_OWN_CAR: dtype: object, unique values: 2. Categories: ['N' 'Y']
FLAG_OWN_REALTY: dtype: object, unique values: 2. Categories: ['Y' 'N']
NAME_TYPE_SUITE: dtype: object, unique values: 7.
NAME_INCOME_TYPE: dtype: object, unique values: 8.
NAME_EDUCATION_TYPE: dtype: object, unique values: 5.
NAME_FAMILY_STATUS: dtype: object, unique values: 6.
NAME_HOUSING_TYPE: dtype: object, unique values: 6.
OCCUPATION_TYPE: dtype: object, unique values: 18.
WEEKDAY_APPR_PROCESS_START: dtype: object, unique values: 7.
ORGANIZATION_TYPE: dtype: object, unique values: 58.
FONDKAPREMONT_MODE: dtype: object, unique values: 4. Categories: ['reg oper account' nan 'org spec account' 'reg oper spec account'
 'not specified']
HOUSETYPE_MODE: dtype: object, unique values: 3. Categories: ['block of flats' nan 'terraced house' 'specific ho

Some values look more like boolean than categorical, so we will be transforming these columns as well.

In [8]:
def to_bool(df, col_name):
    df[col_name] = df[col_name].map({"N": False, "Y": True,
                                     "No": False, "Yes":True,
                                     0: False, 1: True}).astype(bool)
    return df

In [9]:
for col in ["FLAG_OWN_CAR", "FLAG_OWN_REALTY", "EMERGENCYSTATE_MODE"]:
  full_loans = to_bool(full_loans, col)

There's also many "FLAG" and other columns that look like boolean. We will transform them as well to optimize our dataset.

In [10]:
should_be_bool = []

for col in full_loans.columns:
  if hl.is_binary(full_loans[col]):
    should_be_bool.append(col)

should_be_bool

['TARGET',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EMERGENCYSTATE_MODE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [11]:
full_loans[should_be_bool].isna().any().sort_values(ascending=False).head(3)

EMERGENCYSTATE_MODE     True
TARGET                 False
FLAG_DOCUMENT_5        False
dtype: bool

In [12]:
full_loans["EMERGENCYSTATE_MODE_MISSING"] = full_loans["EMERGENCYSTATE_MODE"].isna()

full_loans[should_be_bool] = full_loans[should_be_bool].astype("bool")

In [13]:
full_loans.dtypes

SK_ID_CURR                         int64
TARGET                              bool
NAME_CONTRACT_TYPE              category
CODE_GENDER                     category
FLAG_OWN_CAR                        bool
FLAG_OWN_REALTY                     bool
CNT_CHILDREN                       int64
AMT_INCOME_TOTAL                 float64
AMT_CREDIT                       float64
AMT_ANNUITY                      float64
AMT_GOODS_PRICE                  float64
NAME_TYPE_SUITE                 category
NAME_INCOME_TYPE                category
NAME_EDUCATION_TYPE             category
NAME_FAMILY_STATUS              category
NAME_HOUSING_TYPE               category
REGION_POPULATION_RELATIVE       float64
DAYS_BIRTH                         int64
DAYS_EMPLOYED                      int64
DAYS_REGISTRATION                float64
DAYS_ID_PUBLISH                    int64
OWN_CAR_AGE                      float64
FLAG_MOBIL                          bool
FLAG_EMP_PHONE                      bool
FLAG_WORK_PHONE 

In [14]:
full_loans["HOUR_APPR_PROCESS_START"].describe(

)

count   307511.000
mean        12.063
std          3.266
min          0.000
25%         10.000
50%         12.000
75%         14.000
max         23.000
Name: HOUR_APPR_PROCESS_START, dtype: float64

## Dealing with Strange Values

"Days" Columns have negative values to indicate an event before the dataset was saved. However, one column "DAYS_EMPLOYED" has anomaly values of "365243" which are positive, and equal to ~100 years. We will mark these occurrences as anomalies in separate column, and convert these values to nulls.

In [15]:
full_loans['DAYS_EMPLOYED_ANOMALY'] = full_loans["DAYS_EMPLOYED"] == 365243
full_loans['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

There are also some XNA and XAP values, but we will leave them as is for now.

## Splitting the Dataset

90/5/5 split is used for **training** / **validation** / **test** sets, as we have large enough dataset to move away from the more standard 60/20/20 split. This gives us ~15k rows (and over 1k defaults) in validation and test sets each, which should be enough representation for possible feature combinations.

In [16]:
X_train, X_vt, y_train, y_vt = train_test_split(
    full_loans.drop(columns=["TARGET"]),
    full_loans["TARGET"],
    test_size=0.1,
    random_state=42,
    stratify=full_loans["TARGET"]
    )

X_valid, X_test, y_valid, y_test = train_test_split(
    X_vt,
    y_vt,
    test_size=0.5,
    random_state=42,
    stratify=y_vt
    )


print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(276759, 123)
(15376, 123)
(15376, 123)


In [17]:
X_train, X_vt, y_train, y_vt = train_test_split(
    full_loans,
    full_loans["TARGET"],
    test_size=0.1,
    random_state=42,
    stratify=full_loans["TARGET"]
    )

X_valid, X_test, y_valid, y_test = train_test_split(
    X_vt,
    y_vt,
    test_size=0.5,
    random_state=42,
    stratify=y_vt
    )

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(276759, 124)
(15376, 124)
(15376, 124)


In [18]:
X_train.to_pickle(dir_path + "data/train.pkl")
X_valid.to_pickle(dir_path + "data/valid.pkl")
X_test.to_pickle(dir_path + "data/test.pkl")

# Previous Loans

Since we are done with the main file, let's move on to the additional files we have. We'll begin with previous applications.

The following file contains all previous applications for HomeCredit loans. We can assume that HomeCredit inspects previous credit before approving _current_ applications.

In [40]:
prev = pd.read_csv(dir_path + "original_data/previous_application.csv")
print(prev.shape)
prev.head()

(1670214, 37)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.000,17145.000,0.000,17145.000,SATURDAY,15,Y,1,0.000,0.183,0.867,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.000,middle,POS mobile with interest,365243.000,-42.000,300.000,-42.000,-37.000,0.000
1,2802425,108129,Cash loans,25188.615,607500.000,679671.000,NaN,607500.000,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.000,low_action,Cash X-Sell: low,365243.000,-134.000,916.000,365243.000,365243.000,1.000
2,2523466,122040,Cash loans,15060.735,112500.000,136444.500,NaN,112500.000,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.000,high,Cash X-Sell: high,365243.000,-271.000,59.000,365243.000,365243.000,1.000
3,2819243,176158,Cash loans,47041.335,450000.000,470790.000,NaN,450000.000,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.000,middle,Cash X-Sell: middle,365243.000,-482.000,-152.000,-182.000,-177.000,1.000
4,1784265,202054,Cash loans,31924.395,337500.000,404055.000,NaN,337500.000,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.000,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
prev.duplicated().sum()

0

There doesn't seem to be direct duplicates. However, there are two columns indicating potential double entries of the applications.

From data dictionary: "Flag if it was last application for the previous contract. Sometimes by mistake of client or our clerk there could be more applications for one single contract".

In [42]:
prev["FLAG_LAST_APPL_PER_CONTRACT"].value_counts()

FLAG_LAST_APPL_PER_CONTRACT
Y    1661739
N       8475
Name: count, dtype: int64

In [43]:
prev[prev["FLAG_LAST_APPL_PER_CONTRACT"]=="N"]["NAME_CONTRACT_STATUS"].value_counts()

NAME_CONTRACT_STATUS
Refused     8473
Canceled       2
Name: count, dtype: int64

"Flag if the application was the last application per day of the client. Sometimes clients apply for more applications a day. Rarely it could also be error in our system that one application is in the database twice"

In [44]:
prev["NFLAG_LAST_APPL_IN_DAY"].value_counts()

NFLAG_LAST_APPL_IN_DAY
1    1664314
0       5900
Name: count, dtype: int64

In [45]:
prev[prev["NFLAG_LAST_APPL_IN_DAY"]==0]["NAME_CONTRACT_STATUS"].value_counts()

NAME_CONTRACT_STATUS
Refused         5150
Approved         737
Unused offer      13
Name: count, dtype: int64

This column looks like it could be informative, so we will not be dropping it.

#### Processing

Below, we do the following:
1. Get rid of duplicate applications;
2. Deal with special values ("XNA" and "XPA" as Missing, in DAYS_ columns - 365243 we replace as -1 days);
3. Encode categorical values
4. Drop string columns, as well as columns that would not make sense.

In [46]:
prev_proc = prev.copy()

# Duplicates
prev_proc = prev_proc[prev_proc["FLAG_LAST_APPL_PER_CONTRACT"]=="Y"]
prev_proc.drop(columns=["FLAG_LAST_APPL_PER_CONTRACT"], inplace=True)

# Missing Value
prev_proc = prev_proc.replace({"XNA": np.nan, "XNA": np.nan})

for col in prev_proc.columns:
    if "DAYS_" in col:
        prev_proc[col] = prev_proc[col].replace({365243: 1})

# Encoding
prev_proc["NAME_YIELD_GROUP"] = prev_proc["NAME_YIELD_GROUP"].replace({
    "low_action": 1, "low_normal": 2, "middle": 3, "high": 4})

prev_proc["NAME_CONTRACT_STATUS"] = prev_proc["NAME_CONTRACT_STATUS"] == "Approved"
prev_proc["NAME_CLIENT_TYPE"] = prev_proc["NAME_CLIENT_TYPE"] == "New"

# Dropping string columns
str_cols = []

for col in prev_proc.columns:
    if not pd.api.types.is_numeric_dtype(prev_proc[col]):
        str_cols.append(col)

prev_proc.drop(columns=str_cols, inplace=True)

irrel_cols = ["HOUR_APPR_PROCESS_START"]
prev_proc.drop(columns=irrel_cols, inplace=True)

print(prev_proc.shape)
prev_proc.head()

(1661739, 23)


,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_CLIENT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,1730.430,17145.000,17145.000,0.000,17145.000,1,0.000,0.183,0.867,True,-73,False,35,12.000,3.000,1.000,-42.000,300.000,-42.000,-37.000,0.000
1,2802425,108129,25188.615,607500.000,679671.000,NaN,607500.000,1,NaN,NaN,NaN,True,-164,False,-1,36.000,1.000,1.000,-134.000,916.000,1.000,1.000,1.000
2,2523466,122040,15060.735,112500.000,136444.500,NaN,112500.000,1,NaN,NaN,NaN,True,-301,False,-1,12.000,4.000,1.000,-271.000,59.000,1.000,1.000,1.000
3,2819243,176158,47041.335,450000.000,470790.000,NaN,450000.000,1,NaN,NaN,NaN,True,-512,False,-1,12.000,3.000,1.000,-482.000,-152.000,-182.000,-177.000,1.000
4,1784265,202054,31924.395,337500.000,404055.000,NaN,337500.000,1,NaN,NaN,NaN,False,-781,False,-1,24.000,4.000,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
prev_proc.to_csv(dir_path + "data/previous_application.csv", index=False)

del prev, prev_proc

## Installment Payments

This file shows repayment history for the previously disbursed credits in HomeCredit.

In [48]:
paym = pd.read_csv(dir_path + "original_data/installments_payments.csv")
print(paym.shape)
paym[paym["SK_ID_PREV"]==2714724].sort_values(by="NUM_INSTALMENT_NUMBER", ascending=True)

(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
2843935,2714724,167756,1.000,1,-1413.000,-1421.000,2165.040,2165.040
4,2714724,167756,1.000,2,-1383.000,-1366.000,2165.040,2160.585
1786772,2714724,167756,1.000,2,-1383.000,-1421.000,2165.040,4.455
422906,2714724,167756,1.000,3,-1353.000,-1366.000,2165.040,8.415
2998490,2714724,167756,1.000,3,-1353.000,-1336.000,2165.040,2156.625
2201494,2714724,167756,2.000,4,-1323.000,-1336.000,9691.290,9691.290


* DAYS_INSTALLMENT - when it was supposed to be paid
* DAYS_ENTRY_PAYMENT - when it was actually paid

In [49]:
paym.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

#### Processing

In [50]:
paym["late"] = paym["DAYS_INSTALMENT"] < paym["DAYS_ENTRY_PAYMENT"]
paym["late"].mean()

0.08428042657471103

Interestingly, the proportion of late payments is similar to proportion of loan defaults.

In [51]:
paym["days_late"] = paym["DAYS_ENTRY_PAYMENT"] - paym["DAYS_INSTALMENT"]
paym["days_late"] = paym["days_late"].clip(lower=0)
paym[paym["SK_ID_PREV"]==2714724].sort_values(by="NUM_INSTALMENT_NUMBER", ascending=True)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late,days_late
2843935,2714724,167756,1.000,1,-1413.000,-1421.000,2165.040,2165.040,False,0.000
4,2714724,167756,1.000,2,-1383.000,-1366.000,2165.040,2160.585,True,17.000
1786772,2714724,167756,1.000,2,-1383.000,-1421.000,2165.040,4.455,False,0.000
422906,2714724,167756,1.000,3,-1353.000,-1366.000,2165.040,8.415,False,0.000
2998490,2714724,167756,1.000,3,-1353.000,-1336.000,2165.040,2156.625,True,17.000
2201494,2714724,167756,2.000,4,-1323.000,-1336.000,9691.290,9691.290,False,0.000


Since we can have two rows for one payment when it's missed, we will be aggregating it to only have one row for one instalment.

In [52]:
paym_proc = paym.groupby(by=["SK_ID_PREV", "NUM_INSTALMENT_NUMBER"]).agg({
    "NUM_INSTALMENT_VERSION":	"mean",
    "DAYS_INSTALMENT": "min",
    "DAYS_ENTRY_PAYMENT": "max",
    "AMT_INSTALMENT": "min",
    "AMT_PAYMENT": "sum",
    "late": "max",
    "days_late": "max"
})

paym_proc = paym_proc.reset_index()

print(paym_proc.shape)
paym_proc[paym_proc["SK_ID_PREV"]==2714724].sort_values(by="NUM_INSTALMENT_NUMBER", ascending=True)

(12861994, 9)


,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late,days_late
11984110,2714724,1,1.000,-1413.000,-1421.000,2165.040,2165.040,False,0.000
11984111,2714724,2,1.000,-1383.000,-1366.000,2165.040,2165.040,True,17.000
11984112,2714724,3,1.000,-1353.000,-1336.000,2165.040,2165.040,True,17.000
11984113,2714724,4,2.000,-1323.000,-1336.000,9691.290,9691.290,False,0.000


In [53]:
paym_proc.to_csv(dir_path + "data/installments_payments.csv", index=False)

del paym, paym_proc

## POS_CASH balance

 This file holds monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with HomeCredit.

 SK_DPD and SK_DPD_DEF columns holds most interesting information, as they indicate if the payments where late.

In [54]:
pos_cash = pd.read_csv(dir_path + "original_data/POS_CASH_balance.csv")
print(pos_cash.shape)
pos_cash.head()

(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.000,45.000,Active,0,0
1,1715348,367990,-33,36.000,35.000,Active,0,0
2,1784872,397406,-32,12.000,9.000,Active,0,0
3,1903291,269225,-35,48.000,42.000,Active,0,0
4,2341044,334279,-35,36.000,35.000,Active,0,0


In [55]:
pos_cash["NAME_CONTRACT_STATUS"].value_counts()

NAME_CONTRACT_STATUS
Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: count, dtype: int64

In [56]:
pos_cash.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [57]:
pos_cash.to_csv(dir_path + "data/POS_CASH_balance.csv", index=False)

del pos_cash

## Credit Card
This file contains monthly balance snapshots credit cards from the previous applications.

Here, we will only be dropping the categorical columns, as we can't aggregate them properly.

In [58]:
credit = pd.read_csv(dir_path + "original_data/credit_card_balance.csv")
print(credit.shape)
credit[credit["SK_ID_PREV"]==2562384].sort_values(by="MONTHS_BALANCE", ascending=True).head(7)

(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1811769,2562384,378907,-48,69186.690,270000,67500.000,67500.000,0.000,0.000,NaN,0.000,0.000,67500.000,67841.190,67841.190,1.000,1,0.000,0.000,NaN,Active,0,0
2584127,2562384,378907,-47,62727.075,270000,0.000,0.000,0.000,0.000,3498.975,4500.000,4500.000,60979.410,63246.825,63246.825,0.000,0,0.000,0.000,1.000,Active,0,0
3284876,2562384,378907,-46,62577.900,270000,0.000,0.000,0.000,0.000,3399.795,3600.000,3600.000,60796.035,63082.080,63082.080,0.000,0,0.000,0.000,2.000,Active,0,0
2653597,2562384,378907,-45,61484.400,270000,0.000,0.000,0.000,0.000,3346.695,3600.000,3600.000,59734.170,61980.255,61980.255,0.000,0,0.000,0.000,3.000,Active,0,0
1039133,2562384,378907,-44,60736.680,270000,0.000,0.000,0.000,0.000,3291.390,3375.000,3375.000,59078.160,61224.345,61224.345,0.000,0,0.000,0.000,4.000,Active,0,0
2021610,2562384,378907,-43,59786.865,270000,0.000,0.000,0.000,0.000,3242.250,3375.000,3375.000,58095.090,60267.735,60267.735,0.000,0,0.000,0.000,5.000,Active,0,0
1764744,2562384,378907,-42,58743.675,270000,0.000,0.000,0.000,0.000,3194.190,3375.000,3375.000,57133.755,59216.985,59216.985,0.000,0,0.000,0.000,6.000,Active,0,0


In [59]:
credit.isna().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [60]:
credit_proc = credit.drop(columns=["NAME_CONTRACT_STATUS"])

In [61]:
credit_proc.to_csv(dir_path + "data/credit_card_balance.csv", index=False)

del credit, credit_proc

# Bureau Data

Now that we looked at previous application related files, let's do the same for bureau data.

This file contains client's previous credits provided by other financial institutions and were reported to Credit Bureau.

In [19]:
bureau = pd.read_csv(dir_path + "original_data/bureau.csv")
print(bureau.shape)
bureau.head()

(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.000,-153.000,NaN,0,91323.000,0.000,NaN,0.000,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.000,NaN,NaN,0,225000.000,171342.000,NaN,0.000,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.000,NaN,NaN,0,464323.500,NaN,NaN,0.000,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.000,NaN,NaN,0.000,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.000,NaN,77674.500,0,2700000.000,NaN,NaN,0.000,Consumer credit,-21,NaN


In [20]:
print(bureau["CREDIT_CURRENCY"].value_counts())

CREDIT_CURRENCY
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: count, dtype: int64


Since the proportions of currencies are so different, and we have no indication about currencies in the main file, we will only keep data on currency 1.

In [21]:
bureau = bureau[bureau["CREDIT_CURRENCY"] == "currency 1"]
print(bureau.shape)

(1715020, 17)


In [22]:
print(bureau["CREDIT_ACTIVE"].value_counts())

CREDIT_ACTIVE
Closed      1078062
Active       630426
Sold           6511
Bad debt         21
Name: count, dtype: int64


Let's encode the **credit status** values, and then drop remaining string columns.

In [23]:
bureau["BAD_DEBT"] = bureau["CREDIT_ACTIVE"] == "Bad debt"
bureau["SOLD"] = bureau["CREDIT_ACTIVE"] == "Sold"
bureau["CREDIT_ACTIVE"] = bureau["CREDIT_ACTIVE"] == "Active"

In [24]:
bureau.drop(columns=["CREDIT_CURRENCY", "CREDIT_TYPE"], inplace=True)

In [25]:
print(bureau.shape)
bureau.head(3)

(1715020, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BAD_DEBT,SOLD
0,215354,5714462,False,-497,0,-153.000,-153.000,NaN,0,91323.000,0.000,NaN,0.000,-131,NaN,False,False
1,215354,5714463,True,-208,0,1075.000,NaN,NaN,0,225000.000,171342.000,NaN,0.000,-20,NaN,False,False
2,215354,5714464,True,-203,0,528.000,NaN,NaN,0,464323.500,NaN,NaN,0.000,-16,NaN,False,False


In [26]:
bureau.to_csv(dir_path + "data/bureau.csv", index=False)

del bureau

## Bureau Balances
This file contains monthly balances of the previous credits in the **bureau** file.

In [62]:
bureau_bal = pd.read_csv(dir_path + "original_data/bureau_balance.csv")
print(bureau_bal.shape)
bureau_bal.head()

(27299925, 3)


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


From the Data Dictionary, we see the following explanation for status:

"Status of Credit Bureau loan during the month (active, closed, DPD0-30,… [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,… 5 means DPD 120+ or sold or written off ] )
* C - closed
* X - status unknown
* 0 - no DPD
* 1 - maximal did during month between 1-30
* 2 - 31-60
* 5 - DPD 120+ or sold or written off"

Furthermore, right away we see many rows with status "Closed" in a row, indicating that they most likely have redundant information. So we will delete these duplicate rows.

In [63]:
bureau_bal[bureau_bal["STATUS"]=="C"].shape

(13646993, 3)

In [64]:
balance_min_c = bureau_bal[bureau_bal["STATUS"]=="C"].groupby(
    "SK_ID_BUREAU").agg({"MONTHS_BALANCE": "min"})

balance_min_c.columns = ["min_Closed"]
balance_min_c.reset_index(inplace=True)
print(balance_min_c.shape)
balance_min_c.head()

(449604, 2)


,SK_ID_BUREAU,min_Closed
0,5001709,-85
1,5001710,-47
2,5001712,-8
3,5001716,-38
4,5001717,-4


In [65]:
bureau_bal = bureau_bal.merge(balance_min_c, how="left", on="SK_ID_BUREAU")
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,min_Closed
0,5715448,0,C,-8.000
1,5715448,-1,C,-8.000
2,5715448,-2,C,-8.000
3,5715448,-3,C,-8.000
4,5715448,-4,C,-8.000


In [66]:
bureau_bal.shape

(27299925, 4)

In [67]:
bureau_bal[
    (bureau_bal["STATUS"]!="C") &
     (bureau_bal["MONTHS_BALANCE"] > bureau_bal["min_Closed"])
     ].shape

(121, 4)

There are only 121 rows that have other values than "Closed" after the loan being marked Closed for the first time. Other than that - the repeating rows of "Closed" status rows are redundant, so we will be dropping them.

In [68]:
bureau_bal = bureau_bal[
    (bureau_bal["STATUS"]!="C") |
     (bureau_bal["MONTHS_BALANCE"] <= bureau_bal["min_Closed"])
    ]

bureau_bal.drop(columns=["min_Closed"], inplace=True)
print(bureau_bal.shape)
bureau_bal.head(4)

(14102536, 3)


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
8,5715448,-8,C
9,5715448,-9,0
10,5715448,-10,0
11,5715448,-11,X


Now, let's transform the values to numeric ones. Furthermore, we will be converting them to DPD (Days Past Due). Since STATUS value of 5 could mean that the loans Bureau Loan was sold of, or was much later than >121 days, we will also make a new column "VERY_LATE".

In [69]:
status_dpd_dict = {
    1.0: (1 + 30) / 2,
    2.0: (31 + 60) / 2,
    3.0: (61 + 90) / 2,
    4.0: (91 + 120) / 2,
    5.0: (121 + 150) / 2,
    }

status_dpd_dict

{1.0: 15.5, 2.0: 45.5, 3.0: 75.5, 4.0: 105.5, 5.0: 135.5}

In [70]:
bureau_bal["DPD"] = bureau_bal["STATUS"].replace({"C": 0, "X": np.nan}).astype(
    "float64").replace(status_dpd_dict)

bureau_bal["VERY_LATE"] = bureau_bal["STATUS"] == "5"
bureau_bal["DPD_ANY"] = bureau_bal["DPD"] > 0

In [71]:
bureau_bal["DPD"].value_counts()

DPD
0.000      7949111
15.500      242347
135.500      62406
45.500       23419
75.500        8924
105.500       5847
Name: count, dtype: int64

In [72]:
bureau_bal[bureau_bal["DPD_ANY"]==True].sample(5, random_state=42)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,DPD,VERY_LATE,DPD_ANY
25837883,5979403,-96,1,15.500,False,True
9272963,5089433,-2,2,45.500,False,True
15062970,6766423,-47,5,135.500,True,True
17566464,6117021,-54,1,15.500,False,True
1431194,5718555,-55,1,15.500,False,True


In [73]:
bureau_bal.drop(columns=["STATUS"], inplace=True)

Finally, let's save the file.

In [74]:
bureau_bal.to_csv(dir_path + "data/bureau_balance.csv", index=False)

del bureau_bal